In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import re
import os

def filter_data(input_file_path, output_file_path,keyword,affirm_report,order_report_table,reg_report):
    
    for file in os.listdir(input_file_path):
        dataFrame= pd.read_csv(os.path.join(input_file_path, file))
        # select rows containing text "Affirm"
        data_filtered = dataFrame.loc[dataFrame['DESCRIPTION'].str.contains(keyword),['DATE', 'DESCRIPTION', 'RECEIVED']]
        # data_filtered.to_csv(os.path.join(output_file_path, file), index = False)

        #split the decription
        new =data_filtered.DESCRIPTION.str.split(expand=True)
        data_filtered['DEPOSIT_ID']=new[15]
        data_filtered.drop(columns =["DESCRIPTION"], inplace = True)
        data_filtered['DESCRIPTION']=new[3]+' '+new[4]

        # split the deposit id
        new_id=data_filtered.DEPOSIT_ID.str.split(":",expand=True)
        data_filtered.drop(columns =["DEPOSIT_ID"], inplace = True)
        data_filtered['DEPOSIT_ID']=new_id[1]
      
        #read the affirm report file
        data_report=pd.read_csv(affirm_report)
        data_report.rename(columns = {'deposit_id':'DEPOSIT_ID'}, inplace = True)
  
        #combine two csv files based on DEPOSIT ID
        df = pd.merge(data_filtered, data_report,on="DEPOSIT_ID")
        # print(df.to_string())
        df.rename(columns = {'RECEIVED':'BANK AMOUNT'}, inplace = True)
        df.rename(columns = {'order_id':'ORDER'}, inplace = True)
        df['FEE']=df['fees']+df['txn_fees']
        df['ORDER']=df['ORDER'].astype(str)

        #read the order report table
        report_table=pd.read_csv(order_report_table)
        report_table.rename(columns = {'Order_Number':'ORDER'}, inplace = True)
        #covert the object type to float
        report_table['Item_Price']=report_table['Item_Price'].apply(lambda x: float(x.split()[0].replace(',', '')))
        report_table['Item_Discount']=report_table['Item_Discount'].apply(lambda x: float(x.split()[0].replace(',', '')))
       
        #combine report table to df
        data_table=pd.merge(df,report_table,on='ORDER')
        data_table.rename(columns = {'Item_Price':'PRICE'}, inplace = True) # rename columns
        data_table.rename(columns = {'Item_Discount':'DISCOUNT'}, inplace = True) # rename columns
        data_table.rename(columns = {'Order_Date':'QBO DATE'}, inplace = True) # rename columns
        data_table['QBO Description']=data_table['ORDER']
        data_table['INCOME']=data_table['PRICE']

        #add new columns
        data_table['PRICE']=data_table['PRICE'].astype(float)
        data_table['DISCOUNT']=data_table['DISCOUNT'].astype(float)
        data_table['NET AMOUNT']=data_table['PRICE']+data_table['FEE']+data_table['DISCOUNT']
        
        #add a new columns check by comparing two columns
        data_table['CHECK']=np.where((data_table['NET AMOUNT'] == data_table['total_settled']), 'Correct', 'Incorrect')
        # print(data_table.to_string())

        #compare the result with registration report to get coursemonth
        registr_report=pd.read_csv(reg_report)
        output_data=data_table[['DATE', 'DESCRIPTION','BANK AMOUNT','PRICE','FEE','DISCOUNT','NET AMOUNT','total_settled','CHECK','ORDER','QBO DATE','QBO Description','INCOME']].copy()
        print(registr_report)
        #convert the output to csv file
        # output_data.to_csv('affirm_summary.csv', encoding = 'utf-8-sig') 
        # report='affirm_summary.csv'
        # output_data.to_csv(os.path.join(output_file_path,report), index = False) 
        
    

In [6]:
filter_data('/content/drive/MyDrive/Colab Notebooks/input_file_path','/content/drive/MyDrive/Colab Notebooks/output_file_path','AFFIRM','/content/drive/MyDrive/Affirm Testing/Affirm_Report_0123.csv','/content/drive/MyDrive/Affirm Testing/order_report_Jan22_Jan23.csv','/content/drive/MyDrive/Colab Notebooks/registration_report_Dec22 _Jan23  - Sheet1.csv')


     Sale Date Registered Date    Order id     Attribute Name  \
0   01-12-2022      01-12-2022  1000039446      B2C Classroom   
1   02-12-2022      02-12-2022  1000039501  Virtual Classroom   
2   02-12-2022      02-12-2022  1000039502             Online   
3   02-12-2022      02-12-2022  1000039505      B2C Classroom   
4   02-12-2022      02-12-2022  1000039511             Online   
..         ...             ...         ...                ...   
59  30-01-2023      30-01-2023  1000045752      B2C Classroom   
60  30-01-2023      30-01-2023  1000045754      B2C Classroom   
61  30-01-2023      30-01-2023  1000045755      B2C Classroom   
62  31-01-2023      31-01-2023  1000045791      B2C Classroom   
63  31-01-2023      31-01-2023  1000045808             Online   

                         Delegate Name        Session ID  \
0   Diana Test - Please disregard Test    OPST3016041023   
1                        Jamil Ahmadov  OPST3016041023-V   
2           Christian Adrian Friedrich 